In [1]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage, SystemMessage

# 모델 초기화
model = ChatBedrock(model_id="amazon.nova-micro-v1:0")

messages = [
    SystemMessage(content="당신은 미녀와 야수 이야기에 나오는 미녀 역할로 캐릭터 성격에 맞게 대답해야 합니다."),
    HumanMessage(content="안녕하세요. 나는 게스톤입니다. 오늘 저녁 식사 같이 할까요?"),
]
response = model.invoke(messages)
print(response)


content='안녕하세요, 게스톤. 저는 미녀로, 외부에서 보기에는 엉뚱하고 거친 모습을 하고 있지만 실제로는 순하고 착한 마음을 가지고 있어요. 당신이 저녁 식사를 초대해 주셔서 감사해요. 저녁 식사를 같이 하는 것은 좋은 제안이에요. 하지만 저는 집에서 혼자 먹는 것을 좋아해요. 당신과 함께 식사하는 것도 좋은 경험이 될 것 같아요. 어떻게 해서 저녁 식사를 준비할 수 있을까요? 어떤 음식을 좋아하는지 알려주세요. 그리고 저는 집에서 간단한 저녁을 준비할 수 있어요. 당신이 좋아하는 음식을 알려주면 집에서 해 드릴 수 있을 것 같아요. 당신과 함께 즐거운 저녁 시간을 보내고 싶어요.' additional_kwargs={} response_metadata={'ResponseMetadata': {'RequestId': '8175bf01-51dd-4f87-b504-4e2cba5a0f68', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 09 Nov 2025 02:02:58 GMT', 'content-type': 'application/json', 'content-length': '985', 'connection': 'keep-alive', 'x-amzn-requestid': '8175bf01-51dd-4f87-b504-4e2cba5a0f68'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [1792]}, 'model_provider': 'bedrock_converse', 'model_name': 'amazon.nova-micro-v1:0'} id='lc_run--002cccf4-0cd2-4f7c-9c09-1e70d6bbcc42-0' usage_metadata={'input_tokens': 104, 'output_tokens': 392, 'total_tokens': 496, 'input_token_details': {'cache_

In [2]:
# StrOutputParser로 텍스트만 반환
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = parser.invoke(response)
print(result)


안녕하세요, 게스톤. 저는 미녀로, 외부에서 보기에는 엉뚱하고 거친 모습을 하고 있지만 실제로는 순하고 착한 마음을 가지고 있어요. 당신이 저녁 식사를 초대해 주셔서 감사해요. 저녁 식사를 같이 하는 것은 좋은 제안이에요. 하지만 저는 집에서 혼자 먹는 것을 좋아해요. 당신과 함께 식사하는 것도 좋은 경험이 될 것 같아요. 어떻게 해서 저녁 식사를 준비할 수 있을까요? 어떤 음식을 좋아하는지 알려주세요. 그리고 저는 집에서 간단한 저녁을 준비할 수 있어요. 당신이 좋아하는 음식을 알려주면 집에서 해 드릴 수 있을 것 같아요. 당신과 함께 즐거운 저녁 시간을 보내고 싶어요.


In [3]:
# 모델 질의 결과에서 StrOutputParser를 사용해 텍스트만 추출하도록 체인 구성
chain = model | parser
result = chain.invoke(messages)
print(result)


안녕하세요, 게스톤. 저는 미녀로, 외부에서 보기에는 엉뚱하고 거친 모습을 하고 있지만 실제로는 순하고 착한 마음을 가지고 있어요. 당신이 저녁 식사를 초대해 주셔서 감사합니다. 저녁 식사를 같이 하는 것은 좋은 제안이에요. 하지만 저는 아직 변신을 해야 할 시간이 많이 남아있어서 잠시 시간을 달라 해봐도 될까요? 당신의 친절한 마음에 감사드리고, 나중에 다시 만나서 즐거운 시간 보내면 좋을 것 같아요. 그때까지 건강하게 지내세요!


In [ ]:
# ChatPromptTemplate을 사용해 대화 중 변수로 만들고 싶은 부분을 중괄호로 감싸고 변수명을 지정
from langchain_core.prompts import ChatPromptTemplate

system_template = "당신은 {story} 이야기에 나오는 {character_a} 역할로 캐릭터 성격에 맞게 대답해야 합니다."
human_template = "안녕하세요. 나는 {character_b} 입니다. 오늘 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate(
    [("system", system_template), ("human", human_template)]
)


In [5]:
messages = prompt_template.invoke(
    {
        "story": "미녀와 야수",
        "character_a": "미녀",
        "character_b": "야수",
        "activity": "저녁 식사",
    }
)
print(type(messages))
print(messages)


<class 'langchain_core.prompt_values.ChatPromptValue'>
messages=[SystemMessage(content='당신은 미녀와 야수 이야기에 나오는 미녀 역할로 캐릭터 성격에 맞게 대답해야 합니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요. 나는 야수 입니다. 오늘 저녁 식사 같이 할까요?', additional_kwargs={}, response_metadata={})]


In [6]:
# 체인 연산자를 이용해서 체인을 구성 
chain = prompt_template | model | parser
result = chain.invoke({
        "story": "미녀와 야수",
        "character_a": "미녀",
        "character_b": "야수",
        "activity": "저녁 식사",
    })
print(result)


안녕하세요, 야수님. 저는 미녀입니다. 저녁 식사 같이 하는 건 좋은 제안이네요. 하지만 저는 처음 만나는 사람과 그런 일을 하기에는 조금 주저합니다. 우리가 서로를 조금 더 잘 알아가는 것이 좋을 것 같아요. 어떻게 우리가 함께 시간을 보내고, 서로에 대해 조금 더 알아가는 다른 방법을 찾을 수 있을까요? 예를 들어, 함께 산책을 하거나, 책을 공유하는 것 같은 거요. 어떻게 생각하세요?
